# Tweets analysis about discrimination against female
Background: Girls have received lots of predjuices during their life time. Like, "because you are a girl, you should be polite, you should be slim, you should get married early and deliver babies." But, girls have some much more potential and should not be labelized like this.

I want to figure out what exactly people are talking about when they comment on female. In this project, I will do the tweet scraping and topic modeling and some other corpus analysis. In the future, I may go further like training my own feminist classification model or even build a dialouge system to help relpy to those uncomfortable comments.

## 1 Tweet Scraping
This is the first step of my project, I will scrape some tweets about 'girls should','women should','girls shouldn't','women shouldn't' to set up my dataset.

Library : snscrape 

Reference : NLP Week 3.2 - Classification Task

In [1]:
#Install the twitter scraping library
!pip install snscrape

In [12]:
import snscrape.modules.twitter as sntwitter

In [13]:
#Initialise data frame
import pandas as pd
dataset = pd.DataFrame(columns = ["text","label"])

### 1.1 scrape key words "womenshould" & "girlsshould"

In [14]:
#How many to grab
maxTweets = 1000

#Keyword search
keyword = 'womenshould' and 'girlsshould'
should_data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

#Set the label for this group
label = "should"
should_tweets = []
#Collect in array and pair each tweet with a class label
for i,tweet in enumerate(should_data):
    if i >= maxTweets :
        break    
    should_tweets = should_tweets + [[tweet.content, label]]

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_5231/1229216043.py:15: FutureWarning: content is deprecated, use rawContent instead
  should_tweets = should_tweets + [[tweet.content, label]]


### 1.2 scarpe key words "women shouldn't" and "girls shouldn't"

In [5]:
#How many to grab
maxTweets = 5000

#Keyword search
keyword = "womenshouldn't" and "girlsshouldn't"
shouldnot_data = sntwitter.TwitterSearchScraper(keyword + '-filter:links -filter:replies').get_items()

#Set the label for this group
label = "shouldn't"
shouldnot_tweets = []
#Collect in array and pair each tweet with a class label
for i,tweet in enumerate(shouldnot_data):
    if i >= maxTweets :
        break    
    shouldnot_tweets = shouldnot_tweets + [[tweet.content, label]]

### 1.3 create dataset

In [15]:
should_raw_data = dataset.append(pd.DataFrame(should_tweets, columns = ["text", "label"]))

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_5231/4218375632.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  should_raw_data = dataset.append(pd.DataFrame(should_tweets, columns = ["text", "label"]))


In [16]:
should_raw_data.head(10)

,text,label
0,The Nigerian men are so fine with the vibes th...,should
1,SOMEPPL THINK LITTLE GIRLSSHOULD BE SEEN AND N...,should
2,It’s certain signs to figure out if ya nigga/g...,should
3,Having your period and working for 10 hours is...,should
4,"Tonight with my girlsshould be fun, always is 😍",should
5,Happy #InternationalWomensDay to all of the st...,should
6,I hope you know that you are beautiful every s...,should
7,"I can't get over him, but he moved on so fast...",should
8,A message to every girl out there:try to disco...,should
9,"""R. Kelly"" isapedophilethatcontinoustoendanger...",should


In [17]:
should_raw_data.shape

(1000, 2)

In [9]:
shouldnot_raw_data = dataset.append(pd.DataFrame(shouldnot_tweets, columns = ["text", "label"]))

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/1491864541.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  shouldnot_raw_data = dataset.append(pd.DataFrame(shouldnot_tweets, columns = ["text", "label"]))


In [10]:
shouldnot_raw_data.head(10)

,text,label
0,#GirlsShouldnt run to another niggas arms when...,shouldn't
1,#GirlsShouldnt date a Dom. If your just trying...,shouldn't
2,#GirlsShouldnt cheat on a guy if they emotiona...,shouldn't
3,#GirlsShouldnt be sucking dick for all types o...,shouldn't
4,#GirlsShouldnt wear pounds of make-up. How can...,shouldn't
5,#GirlsShouldnt wear the same outfit as another...,shouldn't
6,#GirlsShouldnt wake up and not wash that pussy...,shouldn't
7,Some girlsshouldn't wear leggings. . #Squareas...,shouldn't
8,#girlsshouldnt * RT @ChillSach: #GuysShouldnt ...,shouldn't
9,#girlsshouldnt front like they don't like some...,shouldn't


In [11]:
shouldnot_raw_data.shape

(1243, 2)

I scrape 5000 'should't tweets' it turns out 1242 tweets. So maybe tweets contain "womenshouldn't" and "girlsshouldn't" are not that many.

### 1.4 save raw data to file

Tip: Saving raw data to file can save lots of time when I change the parameters later, because I don't need to scrape it again and just need to read the file.

In [12]:
#Write to file
file_name = "should_raw_data"
should_raw_data.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

In [13]:
#Write to file
file_name = "shouldnot_raw_data"
shouldnot_raw_data.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

## 2 Clean data

Because the keyword I use to scrape tweets are quite similar(like 'girlsshould' and 'womenshould'), so there may be some duplicated tweets.  

### 2.1 Find out if there are duplicated tweets

In [14]:
should_dup = should_raw_data[should_raw_data.duplicated()==True]["text"].values
for i in should_dup:
    for j in should_dup:
        if i == j:
            print(i, j)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
shouldnot_dup = shouldnot_raw_data[should_raw_data.duplicated()==True]["text"].values
for i in shouldnot_dup:
    for j in shouldnot_dup:
        if i == j:
            print(i, j)

#GirlsShouldnt suck dick if you aint good at it goofy #GirlsShouldnt suck dick if you aint good at it goofy
4) Openly talk sex or being a player...I personally want a girl who hasnt ever been ran through
#GirlsShouldnt 4) Openly talk sex or being a player...I personally want a girl who hasnt ever been ran through
#GirlsShouldnt
The fact that I don't participate in smoking "cigs" or chewing.... &gt;&gt;&gt;  #girlsshouldnt #trashy #sorrynotsorry The fact that I don't participate in smoking "cigs" or chewing.... &gt;&gt;&gt;  #girlsshouldnt #trashy #sorrynotsorry
Periods are ridiculous girlsshouldn’t be punished for notgetting pregnant. Periods are ridiculous girlsshouldn’t be punished for notgetting pregnant.
#GirlsShouldnt spread rumors about the person they hate. How would you feel if someone did the exact same thing to you? #GirlsShouldnt spread rumors about the person they hate. How would you feel if someone did the exact same thing to you?
#girlsshouldnt depend on guys for things #

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/56133074.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  shouldnot_dup = shouldnot_raw_data[should_raw_data.duplicated()==True]["text"].values


 There are lots duplicated tweets both in 'should_raw_data' and 'shouldnot_raw_data'. 
 
 I need to remove the dulicated tweets.

### 2.2 Remove duplicated tweets and reindex

Reference : https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html

It's necessary to reindex the data after drop the duplicated tweets, if not, it will turn error during the Topic Modelling.

In [16]:
#drop duplicated tweets
should_df = should_raw_data.drop_duplicates(subset=['text'])
#reindex
should_df= should_df.reset_index(drop=True)
print(should_df)
print(should_df.shape)

                                                   text   label
0     SOMEPPL THINK LITTLE GIRLSSHOULD BE SEEN AND N...  should
1     It’s certain signs to figure out if ya nigga/g...  should
2     Having your period and working for 10 hours is...  should
3       Tonight with my girlsshould be fun, always is 😍  should
4     Happy #InternationalWomensDay to all of the st...  should
...                                                 ...     ...
4217     #girlsshould stop playing dumb, it's not cute.  should
4218  #GirlsShould Have More Respect For Them selves...  should
4219  #GirlsShould follow me fuck niggas lol i aint ...  should
4220  #GirlsShould Always respect themselves & act l...  should
4221   #GirlsShould know by now that guys lie. #sadfact  should

[4222 rows x 2 columns]
(4222, 2)


In [17]:
should_text = should_df.loc[:,'text']

In [18]:
#drop duplicated tweets
shouldnot_df = shouldnot_raw_data.drop_duplicates(subset=['text'])
#reindex
shouldnot_df= shouldnot_df.reset_index(drop=True)
print(shouldnot_df)
print(shouldnot_df.shape)

                                                   text      label
0     #GirlsShouldnt run to another niggas arms when...  shouldn't
1     #GirlsShouldnt date a Dom. If your just trying...  shouldn't
2     #GirlsShouldnt cheat on a guy if they emotiona...  shouldn't
3     #GirlsShouldnt be sucking dick for all types o...  shouldn't
4     #GirlsShouldnt wear pounds of make-up. How can...  shouldn't
...                                                 ...        ...
1220  #girlsshouldnt be 5'2  wearing a size 13 in bo...  shouldn't
1221  RT @Dee_Slate: #girlsshouldnt steal sumbody ti...  shouldn't
1222  #girlsshouldnt steal sumbody tim's jus ask me ...  shouldn't
1223  #Girlsshouldnt wear all that black spary cuz w...  shouldn't
1224  RT @KI2daM attn ladies the dance moves done n ...  shouldn't

[1225 rows x 2 columns]
(1225, 2)


In [19]:
shouldnot_text = shouldnot_df.loc[:,'text']

### 2.4 Remove punctuations

In [20]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
new_should_text = should_text
for punctuation in string.punctuation:
    for i, v in new_should_text.iteritems():
        new_should_text[i] = v.replace(punctuation,'')
print(new_should_text)

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/2912699129.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_should_text.iteritems():


0       SOMEPPL THINK LITTLE GIRLSSHOULD BE SEEN AND N...
1       It’s certain signs to figure out if ya niggagi...
2       Having your period and working for 10 hours is...
3          Tonight with my girlsshould be fun always is 😍
4       Happy InternationalWomensDay to all of the str...
                              ...                        
4217           girlsshould stop playing dumb its not cute
4218        GirlsShould Have More Respect For Them selves
4219    GirlsShould follow me fuck niggas lol i aint g...
4220    GirlsShould Always respect themselves  act lik...
4221        GirlsShould know by now that guys lie sadfact
Name: text, Length: 4222, dtype: object


In [22]:
new_shouldnot_text = shouldnot_text
for punctuation in string.punctuation:
    for i, v in new_shouldnot_text.iteritems():
        new_shouldnot_text[i] = v.replace(punctuation,'')
print(new_shouldnot_text)

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/572423519.py:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_shouldnot_text.iteritems():


0       GirlsShouldnt run to another niggas arms when ...
1       GirlsShouldnt date a Dom If your just trying t...
2       GirlsShouldnt cheat on a guy if they emotional...
3       GirlsShouldnt be sucking dick for all types of...
4       GirlsShouldnt wear pounds of makeup How can we...
                              ...                        
1220    girlsshouldnt be 52  wearing a size 13 in boys...
1221    RT DeeSlate girlsshouldnt steal sumbody tims j...
1222    girlsshouldnt steal sumbody tims jus ask me fo...
1223    Girlsshouldnt wear all that black spary cuz wa...
1224    RT KI2daM attn ladies the dance moves done n u...
Name: text, Length: 1225, dtype: object


### 2.5 Remove '/n'

In [23]:
for i, v in new_should_text.iteritems():
        new_should_text[i] = v.replace('\n','')
print(new_should_text)

0       SOMEPPL THINK LITTLE GIRLSSHOULD BE SEEN AND N...
1       It’s certain signs to figure out if ya niggagi...
2       Having your period and working for 10 hours is...
3          Tonight with my girlsshould be fun always is 😍
4       Happy InternationalWomensDay to all of the str...
                              ...                        
4217           girlsshould stop playing dumb its not cute
4218        GirlsShould Have More Respect For Them selves
4219    GirlsShould follow me fuck niggas lol i aint g...
4220    GirlsShould Always respect themselves  act lik...
4221        GirlsShould know by now that guys lie sadfact
Name: text, Length: 4222, dtype: object


/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/1657328449.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_should_text.iteritems():


In [24]:
for i, v in new_shouldnot_text.iteritems():
        new_shouldnot_text[i] = v.replace('\n','')
print(new_shouldnot_text)

0       GirlsShouldnt run to another niggas arms when ...
1       GirlsShouldnt date a Dom If your just trying t...
2       GirlsShouldnt cheat on a guy if they emotional...
3       GirlsShouldnt be sucking dick for all types of...
4       GirlsShouldnt wear pounds of makeup How can we...
                              ...                        
1220    girlsshouldnt be 52  wearing a size 13 in boys...
1221    RT DeeSlate girlsshouldnt steal sumbody tims j...
1222    girlsshouldnt steal sumbody tims jus ask me fo...
1223    Girlsshouldnt wear all that black spary cuz wa...
1224    RT KI2daM attn ladies the dance moves done n u...
Name: text, Length: 1225, dtype: object


/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/3962642740.py:1: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_shouldnot_text.iteritems():


### 2.6 Remove emojis

In [25]:
import re

def remove_emojis(text):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F600-\U0001F64F"  # emoticons
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)

for i, v in new_should_text.iteritems():
    new_should_text[i] = remove_emojis(v)
print(new_should_text)

/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/1086163645.py:17: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_should_text.iteritems():


0       SOMEPPL THINK LITTLE GIRLSSHOULD BE SEEN AND N...
1       It’s certain signs to figure out if ya niggagi...
2       Having your period and working for 10 hours is...
3           Tonight with my girlsshould be fun always is 
4       Happy InternationalWomensDay to all of the str...
                              ...                        
4217           girlsshould stop playing dumb its not cute
4218        GirlsShould Have More Respect For Them selves
4219    GirlsShould follow me fuck niggas lol i aint g...
4220    GirlsShould Always respect themselves  act lik...
4221        GirlsShould know by now that guys lie sadfact
Name: text, Length: 4222, dtype: object


In [26]:
def remove_emojis(text):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F600-\U0001F64F"  # emoticons
                      "]+", re.UNICODE)
    return re.sub(emoj, '', text)

for i, v in new_shouldnot_text.iteritems():
    new_shouldnot_text[i] = remove_emojis(v)

print(new_shouldnot_text)

0       GirlsShouldnt run to another niggas arms when ...
1       GirlsShouldnt date a Dom If your just trying t...
2       GirlsShouldnt cheat on a guy if they emotional...
3       GirlsShouldnt be sucking dick for all types of...
4       GirlsShouldnt wear pounds of makeup How can we...
                              ...                        
1220    girlsshouldnt be 52  wearing a size 13 in boys...
1221    RT DeeSlate girlsshouldnt steal sumbody tims j...
1222    girlsshouldnt steal sumbody tims jus ask me fo...
1223    Girlsshouldnt wear all that black spary cuz wa...
1224    RT KI2daM attn ladies the dance moves done n u...
Name: text, Length: 1225, dtype: object


/var/folders/z5/t7l_xpd11rv10jj2p0x7qzl80000gn/T/ipykernel_8020/3548377863.py:15: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, v in new_shouldnot_text.iteritems():


### 2.7 Save to file

In [27]:
#Write to file
file_name = "new_should_text"
new_should_text.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

In [28]:
#Write to file
file_name = "new_shouldnot_text"
new_shouldnot_text.to_csv("data/" + file_name + ".csv", sep = "\t", columns = ["text"], index = False)

## 3 Topic Modelling

Firstly, I use "nltk.tokenize.casual.TweetTokenizer" to tokenize tweets.

Secondly, I use LiDA to do the Topic Modelling for 'should text'

Thirdly, I use LSA to do the Topic Modelling for 'shouldnot text'


In [29]:
# Required for everyone always:
import pandas as pd 
import numpy as np
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction import _stop_words as stop_words
from sklearn.metrics.pairwise import cosine_similarity as cosine
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.tokenize.casual import casual_tokenize
import pandas as pd
import re
from sklearn.decomposition import TruncatedSVD

### 3.1 Use TweetTokenizer to do the tokenization

Reference : https://www.nltk.org/api/nltk.tokenize.casual.html

In [30]:
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True, reduce_len=True,match_phone_numbers=False)

In [31]:
should_text = new_should_text
print(should_text.shape)

(4222,)


In [32]:
shouldnot_text = new_shouldnot_text
print(shouldnot_text.shape)

(1225,)


In [33]:
should_text_ls = should_text.tolist()
shouldnot_text_ls = shouldnot_text.tolist()

In [34]:
should_str = ''
for i in should_text:
    should_str += i

In [35]:
shouldnot_str = ''
for i in shouldnot_text:
    shouldnot_str += i

In [36]:
tknzr.tokenize(should_str)
tknzr.tokenize(shouldnot_str)

['GirlsShouldnt',
 'run',
 'to',
 'another',
 'niggas',
 'arms',
 'when',
 'things',
 'get',
 'rough',
 'in',
 'the',
 'real',
 'relationship',
 'You',
 'an',
 'immature',
 'little',
 'girl',
 'if',
 'you',
 'doGirlsShouldnt',
 'date',
 'a',
 'Dom',
 'If',
 'your',
 'just',
 'trying',
 'to',
 'escape',
 'the',
 'memories',
 'of',
 'a',
 'guyGirlsShouldnt',
 'cheat',
 'on',
 'a',
 'guy',
 'if',
 'they',
 'emotional',
 'Yall',
 'always',
 'going',
 'gay',
 'just',
 'for',
 'a',
 'girl',
 'who',
 'will',
 'be',
 'just',
 'emotional',
 'dumbassesGirlsShouldnt',
 'be',
 'sucking',
 'dick',
 'for',
 'all',
 'types',
 'of',
 'niggas',
 'Do',
 'it',
 'for',
 'the',
 '1',
 'who',
 'actually',
 'deserves',
 'it',
 'Not',
 'just',
 'for',
 'nut',
 'bubbles',
 'hoGirlsShouldnt',
 'wear',
 'pounds',
 'of',
 'makeup',
 'How',
 'can',
 'we',
 'find',
 'the',
 'face',
 'of',
 'the',
 'real',
 'you',
 'Stop',
 'hiding',
 'behind',
 'tarnishmentsGirlsShouldnt',
 'wear',
 'the',
 'same',
 'outfit',
 'as'

In [37]:
should_token = tknzr.tokenize(should_str)
shouldnot_token = tknzr.tokenize(shouldnot_str)

### 3.2 LDiA Topic Modelling for 'should_text'

Reference : NLP Week 2.2 Lecture--LDiA

#### Clean tokens

Reference : NLP Week 2.1-Text as Numbers

Use 3 method to clean tokens : Capitalisation, Stemming, Lemmatisation

##### Capitalisation

In [38]:
lower_tokens = []
for word in should_token:
    lower_tokens.append(word.lower())
lower_vocab = np.unique(lower_tokens)
print("unique words", lower_vocab.shape)

unique words (6854,)


##### Stemming

In [39]:
#Looking at our hacking book
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
stem_tokens = []
for t in lower_vocab:
    stem_tokens.append(stemmer.stem(t))
stem_vocab = np.unique(stem_tokens)
print("unique words", stem_vocab.shape)

unique words (6196,)


#####  Lemmatisation 

In [40]:
# import nltk
nltk.download("wordnet")
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lem = WordNetLemmatizer()

#Looking at our hacking book
lem_tokens = []
for t in stem_vocab:
    lem_tokens.append(lem.lemmatize(t))
lem_vocab = np.unique(lem_tokens)
print("unique words", lem_vocab.shape)

[nltk_data] Downloading package wordnet to /Users/wangkun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/wangkun/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


unique words (6180,)


##### Define funciton 

In [41]:
from nltk.stem.porter import PorterStemmer

def tokeniser(text):
    lower_tokens = []
    for word in text:
        lower_tokens.append(word.lower())
    lower_vocab = np.unique(lower_tokens)
    
    stemmer = PorterStemmer()
    stem_tokens = []
    for t in lower_vocab:
        stem_tokens.append(stemmer.stem(t))
    stem_vocab = np.unique(stem_tokens)
    
    for t in stem_vocab:
        lem_tokens.append(lem.lemmatize(t))
    lem_vocab = np.unique(lem_tokens)

    return lem_vocab

#### LatentDirichletAllocation



In [42]:
from sklearn.decomposition import LatentDirichletAllocation

In [43]:
#We calculate LDA on the Bag Of Words, NOT TFIDF
count_vectoriser = CountVectorizer(tokenizer=tokeniser)
bag_of_words = count_vectoriser.fit_transform(should_text)
vocab = count_vectoriser.get_feature_names_out()

print(bag_of_words.todense().shape)

(4222, 6246)


In [44]:
#How many topics?
num_topics = 3
pd.options.display.max_columns=num_topics
labels = ['topic{}'.format(i) for i in range(num_topics)]

In [45]:
lda = LatentDirichletAllocation(n_components=num_topics,
                                random_state=123,
                                learning_method='batch')

In [46]:
#May take some time depending on size of dataset!
lda_topics = lda.fit_transform(bag_of_words)

In [47]:
#How much does each topic apply to each song?
lda_topic_vectors_df = pd.DataFrame(lda_topics, index=should_text.values,columns=labels)
lda_topic_vectors_df.sample(10)

,topic0,topic1,topic2
«va2dc «REALSHITZO GirlsShould stop talking down on other females help them out admirers are better than haters»»,0.335051,0.334325,0.330623
girlsshould let me smell their vag when I ask instead of acting all stuck up,0.333494,0.329921,0.336585
girlsshould stop taking pictures and poke their a out 5 feet away from their body to make it look bigger,0.335267,0.333145,0.331588
GirlsShould learn how to take care of themselves in public,0.331693,0.329414,0.338893
HiItsMeCaithlyn GirlsShould be able to eat what they want without feeling fat,0.335051,0.334325,0.330623
GirlsShould be honest If you want a bad boy say that shit dont say you want someone nice the friend zone them,0.335259,0.340834,0.323907
GirlsShould GetOverIt,0.332134,0.329626,0.338240
GirlsShould watch Regular Show,0.332134,0.329626,0.338240
GirlsShould spend a day at the best spa you can afford pumped,0.335051,0.334325,0.330623
GirlsShould learn to say NO It’s important to learn to say NO when people ask us to do things which are not in our best interests DRT,0.331693,0.329414,0.338893


In [48]:
#Highest scoring songs for each topic
for topic in labels:
    print(topic)
    #Get last songs 10 in list (highest scoring for each topic)
    print(should_text.values[lda_topic_vectors_df[topic].argsort().values[-10:]])

topic0
['GirlsShould Learn How To Be Real'
 'GirlsShould Learn How NOT To Start Drama'
 'GirlsShould Learn How To Play Call Of Duty'
 'GirlsShould Learn How To Rock Their Hips '
 'GirlsShould Stop Being So Stuck Up'
 'GirlsShould never let da pass heartbreak fck up there future love'
 'GirlsShould Open Their Eyes And Realize Its Not Always About What You Want But What You Need'
 'GirlsShould Stop Looking For Attention All The Time'
 'girlsshould get their mind straight if you want a true relationship marry god If u want a relationship of sex be a prostitute'
 'GirlsShould Not give a fuck']
topic1
['“bocaprncess GirlsShould ALWAYS be treated like princesses ”'
 '“TheHappyBoner GirlsShould HoldMaDiiiiiiick” buck12288 haha'
 'GirlsShould Let their ex know what he missed out on Soon hell realize that you were the best he ever had '
 'GirlsShould Never be the loudest thing in a restaurant when youre sitting with your so called man BirdAssBitches'
 'GirlsShould have lots of sex'
 'GirlsShoul

In [49]:
#Most relevant tokens for each topic
num_terms = 10
for i, topic in enumerate(lda.components_):
    print("topic " + str(i) + ":")
    #Get last n tokens (highest values)
    print(vocab[topic.argsort()[-num_terms:]])

topic 0:
['selfrespectgirlsshould' 'uggsgirlsshould' 'otherwis' 'oftenambwomb'
 'tryin' 'dontcutit' 'anyth' 'bed' 'gabbyi' 'appli']
topic 1:
['uplik' 'louder' 'pimpin' 'shoesbootsshirt' 'rainbow' 'woth' 'doubl'
 'arrivesgirlsshouldgirlsshould' 'broke' 'daysgirlsshould']
topic 2:
['issuesgirlsshould' 'exxxtra' 'thesexyscorpian' 'marv' 'trippyiesha'
 'braingirlsshould' 'itsom' 'impress' 'exceric' 'steak']


### 3.3 LSA Topic Modelling for 'shouldnot_text'

Reference: NLP Week 2.2 Lecture--LSA

#### TFIDF

In [50]:
from sklearn.decomposition import TruncatedSVD

In [51]:
tfidf_vectoriser = TfidfVectorizer(tokenizer=tokeniser)

In [52]:
#Get TFIDF
tfidf = tfidf_vectoriser.fit_transform(shouldnot_text)
#Save list of unique tokens (vocab) for later
vocab = tfidf_vectoriser.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf.todense(), columns = vocab)
print(tfidf.todense().shape)

(1225, 6252)


In [53]:
#Subtract mean
tfidf_df = tfidf_df - tfidf_df.mean()

In [54]:
#How many topics?
num_topics = 3
pd.options.display.max_columns=num_topics
labels = ['topic{}'.format(i) for i in range(num_topics)]

In [55]:
#Calculate topics
svd = TruncatedSVD(n_components = num_topics, n_iter = 100) 
svd_topic_vectors = svd.fit_transform(tfidf_df.values)

In [56]:
#How much does each topic apply to each token?
topic_weights = pd.DataFrame(svd.components_.T, index=vocab, columns=labels)
topic_weights.sample(20) #display it

,topic0,topic1,topic2
pretti,-0.000503,-0.000313,0.000069
putmeincoach,-0.000503,-0.000313,0.000069
continu,-0.000503,-0.000313,0.000069
threat,-0.000503,-0.000313,0.000069
krusti,-0.000503,-0.000313,0.000069
whydopeoplethink,-0.000503,-0.000313,0.000069
peni,-0.000503,-0.000313,0.000069
hip,-0.000503,-0.000313,0.000069
shuddupgirlsshould,-0.000503,-0.000313,0.000069
xlecutieexrt,-0.000503,-0.000313,0.000069


In [57]:
#How much does each topic apply to each song?
svd_topic_vectors_df = pd.DataFrame(svd_topic_vectors, index=shouldnot_text.values, columns=labels)
svd_topic_vectors_df.sample(10)

,topic0,topic1,topic2
girlsshouldnt be 52 wearing a size 13 in boys shoes,0.067224,0.050814,-0.010777
Girlsshouldnt think jus becuase the las nigga played yu that ima do the same smh ihatethat,0.041571,-0.018897,-0.002481
girlsshouldnt go on like you run the world beyonce lied,0.023706,-0.012541,-0.001986
Nahh thts a bop lol RT Bdavishooper12 girlsshouldnt be hella cocky thats a turnoff,0.041571,-0.018897,-0.002481
GirlsShouldnt let guys talk to them any kind of way when there in a relationship,-0.012209,0.001281,0.000129
GirlsShouldnt Ever Walk Around Watcha Body Count Nigga Im Double Digits,-0.011729,0.001069,0.000014
girlsshouldnt be so stupid,-0.011729,0.001069,0.000014
RT TeemoneeNYC girlsshouldnt wear makeup thats called FALSE ADVERTISING,0.041571,-0.018897,-0.002481
lmaooooooo YoCk RT MoovveeBITCH GirlsShouldnt Wear Open Toe Sandals Wit 6 Toes,0.041571,-0.018897,-0.002481
GirlsShouldntGossip so muchI hate that shit,-0.012209,0.001281,0.000129


In [60]:
#Highest scoring tweets for each topic
for topic in labels:
    print("________" + topic)
    #Get last reviews 10 in list (highest scoring for each topic)
    print(shouldnot_text.values[svd_topic_vectors_df[topic].argsort().values[-10:]])

________topic0
['RT TheRajGuy RT Addicted2Fly167 GirlsShouldnt wear Butt pads smdh'
 'RT Addicted2Fly167 GirlsShouldnt wear Butt pads smdh'
 'GirlsShouldnt wears sandals with corns  hammer toes 0o'
 'GirlsShouldnt give head and immediately go kiss the child good night smh brush ya teeth  least lmao'
 'RT PepperCompinc GirlsShouldnt go mac they should just get there mac done by PepperCompinc'
 'GirlsShouldnt go mac they should just get there mac done by PepperCompinc'
 'RT LIVINGLEGEND17 GirlsShouldnt Rock a weave until there roots puff up and look like an afro lol get that shit done MAindeed'
 'RT TheRajGuy GirlsShouldnt GirlsShouldnt GirlsShouldnt GirlsShouldnt GirlsShouldnt GirlsShouldnt GirlsShouldnt GirlsShouldnt'
 'GirlsShouldnt Be a hoe in there hood then go 2 another borough and get wifed I feel bad for that poor guy thats kissin on u and my kids'
 'RT KI2daM attn ladies the dance moves done n ur room while sippin wit ya girlsshouldnt b takn 2 da club leave dat shit  home please

In [61]:
#Most relevant words for each topic
num_terms = 10
for i in range(num_topics):
    print("___topic " + str(i) + "___")
    topicName = "topic" + str(i)
    weightedlist = topic_weights.get(topicName).sort_values()[-num_terms:]
    print(weightedlist.index.values)

___topic 0___
['02' '008girlsshould' '008' '00' '\ue407' '•' '¤' '\ue105' '\ue412'
 '\ue40e']
___topic 1___
['livesgirlsshould' 'shelleymilano' 'drive' '�' '02' '008' '00' '0' '•'
 '¤']
___topic 2___
['gaythat' 'gentleman' '\ue319' '02' 'livesgirlsshould' 'shelleymilano'
 'drive' '�' '00' '¤']
